In [18]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda, merge, Dense, Flatten
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, Sequential, model_from_json, load_model
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from keras import backend as K
import torch

%run data_loader.ipynb
#%run encoder.ipynb

In [19]:
train_prepared_data = prepare_data(X)
train_prepared_data = np.array(train_prepared_data)
input_x  = train_prepared_data/np.mean(train_prepared_data)

In [20]:
class_names = ['Genuine','Forged']

In [21]:
clf = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])

In [22]:
clf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              #  loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [24]:
#clf.fit(input_x, T, epochs=500, batch_size=50)
clf.fit(input_x, T, epochs=100)

Epoch 1/100
46061/46061 [==============================] - 17s 366us/sample - loss: 0.1432 - acc: 0.9455
Epoch 2/100
46061/46061 [==============================] - 17s 366us/sample - loss: 0.1103 - acc: 0.9582
Epoch 3/100
46061/46061 [==============================] - 16s 348us/sample - loss: 0.1036 - acc: 0.9615
Epoch 4/100
46061/46061 [==============================] - 16s 345us/sample - loss: 0.0931 - acc: 0.9655
Epoch 5/100
46061/46061 [==============================] - 18s 395us/sample - loss: 0.0886 - acc: 0.9685 - loss: 0.0886 - acc: 0.9
Epoch 6/100
46061/46061 [==============================] - 17s 368us/sample - loss: 0.0814 - acc: 0.9703 - loss: 0.0814 - a
Epoch 7/100
46061/46061 [==============================] - 16s 356us/sample - loss: 0.0759 - acc: 0.9721
Epoch 8/100
46061/46061 [==============================] - 17s 375us/sample - loss: 0.0667 - acc: 0.9757
Epoch 9/100
46061/46061 [==============================] - 17s 372us/sample - loss: 0.0660 - acc: 0.9757 - loss: 0.

46061/46061 [==============================] - 14s 311us/sample - loss: 0.0047 - acc: 0.9987
Epoch 73/100
46061/46061 [==============================] - 15s 315us/sample - loss: 0.0266 - acc: 0.9925
Epoch 74/100
46061/46061 [==============================] - 14s 312us/sample - loss: 0.0119 - acc: 0.9967
Epoch 75/100
46061/46061 [==============================] - 14s 314us/sample - loss: 0.0133 - acc: 0.9962
Epoch 76/100
46061/46061 [==============================] - 14s 314us/sample - loss: 0.0154 - acc: 0.9955
Epoch 77/100
46061/46061 [==============================] - 14s 312us/sample - loss: 0.0093 - acc: 0.9974
Epoch 78/100
46061/46061 [==============================] - 14s 313us/sample - loss: 0.0238 - acc: 0.9932
Epoch 79/100
46061/46061 [==============================] - 14s 314us/sample - loss: 0.0061 - acc: 0.9984 - loss: 0.0055 - acc:
Epoch 80/100
46061/46061 [==============================] - 15s 318us/sample - loss: 0.0194 - acc: 0.9949
Epoch 81/100
46061/46061 [===========

In [25]:
#
test_data = pd.read_csv("data/validation_data.csv")
test_data = test_data.iloc[np.random.permutation(len(test_data))]
#test_data = test_data[:20]
test_X = test_data.iloc[:,:-1].values
test_T = test_data.iloc[:,-1].values

test_X[:,0] = 'data/train/'+test_X[:,0]
test_X[:,1] = 'data/train/'+test_X[:,1]

test_prepared_data = prepare_data(test_X)

test_prepared_data = np.array(test_prepared_data)
test_prepared_data  = test_prepared_data/np.mean(test_prepared_data)

test_loss, test_acc = clf.evaluate(test_prepared_data, test_T)
print('Test accuracy:', test_acc)

23205/23205 [==============================] - 2s 74us/sample - loss: 0.0316 - acc: 0.9922
Test accuracy: 0.99215686


In [26]:
test_predictions = clf.predict(test_prepared_data)
print(test_T)

prediction = []
for i in range(test_predictions.shape[0]):
    prediction.append(np.argmax(test_predictions[i]))
    
print(np.array(prediction) )

[1 0 1 ... 0 0 0]
[1 0 1 ... 0 0 0]


In [27]:
# serialize model to JSON
model_json = clf.to_json()
with open("checkpoint/model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
clf.save_weights("checkpoint/model.h5")
print("Saved model to disk")

Saved model to disk


In [28]:
clf.save('checkpoint/trained_model.h5')  # creates a HDF5 file 'my_model.h5'
print("Saved model to disk")

# returns a compiled model
# identical to the previous one


Saved model to disk
